In [60]:
### @author     Peitsa Rautio
### @version    01.08.2024 

# Summary:

# This .ipynb program imports exported Fluidit format data, builds a pandapipes
# network model and exports it as JSON to ./data/models/.

# In general, dashes in raw node names are converted to underscores due to
# Python syntax not supporting dashes in variable names.
# Dash is strictly an operator type in Python.

# Pandapipes and Pandapower © Copyright 2020-2024 by Fraunhofer Institute for
# Energy Economics and Energy System Technology (IEE), Kassel,
# and University of Kassel.

In [61]:
# Optional: check if venv is activated

import sys

def is_venv():
    return (hasattr(sys, 'real_prefix') or
            (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix))

if is_venv():
    print('inside virtualenv or venv')
else:
    print('outside virtualenv or venv')

outside virtualenv or venv


In [62]:
import pandas as pd
import pandapipes as pp
import numpy as np

## Simulation variables and options

# Ambient temperature [K] = t[C] + 273.15
temp_ext_c = 20
temp_ext_k = temp_ext_c + 273.15

# Initial network fluid temperature (flow side) [K] = t[C] + 273.15
t_net_flow_init_c = 105
t_net_flow_init_k = t_net_flow_init_c + 273.15

# Initial network fluid temperature (return side) [K] = t[C] + 273.15
t_net_return_init_c = 35
t_net_return_init_k = t_net_return_init_c + 273.15

# Initial junction and network pressure, flow side [bar]
net_flow_p_bar = 10

# Initial junction and network pressure, return side [bar]
net_return_p_bar = 8

In [63]:
## Import data from Data.xls into Pandas dataframes

# Define data file path
sourcefile = './Data.xlsx'

# Import data
df_heater = pd.read_excel(sourcefile, sheet_name=0)
df_sink = pd.read_excel(sourcefile, sheet_name=1)
df_connection = pd.read_excel(sourcefile, sheet_name=2)
df_nodetype = pd.read_excel(sourcefile, sheet_name=3)

# Prepare blank network for elements
net = pp.create_empty_network(name="Data", fluid="water")
g={}

In [64]:
def createJunction():
    ## Create pandapipes junctions from imported data.
    # Creates separate networks for supply and return lines. Supply and return
    # lines join at sinks and heat sources.

    # Get number of nodes from dataframe
    num_nodes = len(df_nodetype)

    # tracker tracks number of junctions created
    junction_tracker = 0

    # Iterate over all nodes to find and create junctions
    for i in range(num_nodes):

        if df_nodetype.at[i, 'Node Type'] == "Junction":
            
            # Replace dashes with underscores to conform with Python markup
            junction_name = str(df_nodetype.at[i, 'Name']).replace("Junction-",
                                                                "Junction_")
            
            junction_pos_x = float(df_nodetype.at[i, 'X-Coordinate'])
            junction_pos_y = float(df_nodetype.at[i, 'Y-Coordinate'])
            junction_pos_flow = (junction_pos_x, junction_pos_y)
            junction_pos_return = (junction_pos_x, junction_pos_y-100)

            # Define supply and return flow network junctions
            g['%s_supply' % junction_name] = pp.create_junction(net,
                pn_bar=net_flow_p_bar,
                tfluid_k=t_net_flow_init_k,
                geodata=junction_pos_flow,
                name=junction_name + '_supply')
            g['%s_return' % junction_name] = pp.create_junction(net,
                pn_bar=net_return_p_bar,
                tfluid_k=t_net_return_init_k,
                geodata=junction_pos_return,
                name=junction_name + '_return')

            junction_tracker += 2

    #print("Number of junctions created:", junction_tracker)

    # List junctions
    net.junction

> &#9658; Information on multiple source heating network models

JanStock1 commented on Feb 26 2024

Hello everyone,
I would like to comment on this issue as I came up with the same problem of several producers in a district heating network. The solution proposed by @cvTHM worked well for me (additional producers modelled by heat_exchanger and flow_controller). However, since the heat supply and the mass flow are prescribed in this modelling approach (interpreted as base load producer), the supply temperature at these producers is not fixed but depends on the return temperature (temperature difference is fixed). Therefore, the supply temperature at such a producer varies depending on the return temperature, i.e. on the load situation in the district heating network. Thus, the supply temperature at the producers modelled with heat_exchanger and flow_controller is quite low at some load times.

I have tested an alternative approach that ensures a constant supply temperature at the additional producers, but the actual heat supply at these producers differs slightly from the calculated value.
The first producer in the district heating network is still modelled as a circ_pump_pressure component. The circ_pump_mass component, which prescribes a certain mass flow, can be used (since pandapipes 0.9.0) to specify the temperature at the supply node of additional producers. A heat flow can be calculated using the specified mass flow and the known design temperature difference between supply and return (later more on that). The pressure at the supply node of the circ_pump_mass component must be specified, otherwise there may be no mass flow at the supply node (depending on the pressure conditions in the network). However, if a pressure value is set at the supply nodes of the additional producers (circ_pump_mass) and at the first producer (circ_pump_pressure), the resulting mass flow in the network may not be as desired, as the mass flow follows the fixed pressure conditions in the network. Therefore, a flow_controller is required downstream of the supply node of a circ_pump_mass component, which regulates the pressure conditions and ensures the desired mass flow, which is also set to the circ_pump_mass component but is not yet reached due to the first producer, i.e. the fixed pressure at the circ_pump_pressure component.

The main advantage of this proposed modelling approach (circ_pump_mass and flow_controller) is that the supply temperature at the additional producers can be set. However, as the return temperature at the producers is not known in advance, but the supply temperature and the mass flow are specified, the actually supplied heat at these producers deviates slightly from the calculated value. By varying the set mass flows at the producers depending on the supply condition and the estimated heat losses in the network, the actually supplied heat can be approximated to the aimed value.

In [65]:
def createSource():
    ## Create pandapipes source nodes

    # See markdown notes above for information on multiple producer networks.

    # Raw source heater values are given as kW [J/s * 10^3], pp circ_pump_mass
    # uses mass flow mkg/s, use conversion ratio variable to adjust.

    # Conversion ratio from raw data, denotes average ratio between base demand
    # [kW] and base demand [kg/s].  Assumes ~50 K / kg of energy heat difference
    # between flow and return networks (210.4648 * 10^3 J/kg / 4190 J/kg ~= 50).
    e_flow_conv_ratio = (7 / 0.033 + 40.3 / 0.193) / 2 # = 210.4648

    # Output water temperature [K] = t[C] + 273.15
    t_out_c = 110
    t_out_k = t_out_c + 273.15

    # Pressure at flow side of producers [bar]
    source_flow_bar = 10

    # Pressure lift induced by sources [bar] - Used to model pressure difference
    # at pump nodes between input and output
    pressure_lift = 3

    # Production multiplier - general multiplier for testing effects of increasing/
    # reducing heated water production. Multiplier applies to mdot_flow_kg_per_s
    prod_multiplier = 1

    # Get source connections and create source nodes
    for i in range(len(df_heater)):
        
        if str(df_heater.iloc[i,0]) != "Ericsson": # Ericsson special case below
            source_name = str(df_heater.iloc[i,0])
            # Define supply and return flow network sources
            source_row=df_nodetype.loc[df_nodetype['Name']==source_name]
            source_pos_flow=(source_row['X-Coordinate'].values[0],source_row['Y-Coordinate'].values[0])
            source_pos_return=(source_row['X-Coordinate'].values[0],source_row['Y-Coordinate'].values[0]-100)
            g['%s_supply' % source_name] = pp.create_junction(net,
                pn_bar=net_flow_p_bar,
                tfluid_k=t_net_flow_init_k+5,
                geodata=source_pos_flow,
                name=source_name + '_supply')
            g['%s_return' % source_name] = pp.create_junction(net,
                pn_bar=net_return_p_bar-1,
                tfluid_k=t_net_return_init_k,
                geodata=source_pos_return,
                name=source_name + '_return')
            
            source_flow=g[source_name+'_supply']
            source_return=g[source_name+'_return']
            
            pp.create_circ_pump_const_pressure(net,
                                                flow_junction=source_flow,
                                                return_junction=source_return,
                                                plift_bar=pressure_lift,
                                                p_flow_bar=source_flow_bar,
                                                mdot_flow_kg_per_s=pd.to_numeric(df_heater.iloc[i,2])/e_flow_conv_ratio,                
                                                t_flow_k=t_out_k,
                                                name=source_name)
            
            for j in range(len(df_connection)):
                
                if str(df_connection.at[j, 'Start Node']) == source_name:
                    
                    junction_flow = g[str(df_connection.at[j, 'End Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_supply']
                    
                    junction_return = g[str(df_connection.at[j, 'End Node'])
                                            .replace("Junction-", "Junction_") +
                                            '_return']
                    
                    pp.create_pipe_from_parameters(net,
                                                from_junction=source_flow,
                                                to_junction=junction_flow,
                                                length_km=float(str(df_connection.at[j, 'Length [m]']))/1000,
                                                diameter_m=float(str(df_connection.at[j, 'Diameter [mm]']))/1000,
                                                alpha_w_per_m2k=float(str(df_connection.at[j, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                                                text_k=int(temp_ext_k),
                                                name=str(df_connection.at[j,'Name'])+'_supply')

                    pp.create_pipe_from_parameters(net,
                                                from_junction=junction_return,
                                                to_junction=source_return,
                                                length_km=float(str(df_connection.at[j, 'Length [m]']))/1000,
                                                diameter_m=float(str(df_connection.at[j, 'Diameter [mm]']))/1000,
                                                alpha_w_per_m2k=float(str(df_connection.at[j, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                                                text_k=int(temp_ext_k),
                                                name=str(df_connection.at[j,'Name'])+'_supply')
            
                elif str(df_connection.at[j, 'End Node']) == source_name:

                    junction_flow = g[str(df_connection.at[j, 'Start Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_supply']
                    
                    junction_return = g[str(df_connection.at[j, 'Start Node'])
                                            .replace("Junction-", "Junction_") +
                                            '_return']
                    
                    pp.create_pipe_from_parameters(net,
                                                from_junction=source_flow,
                                                to_junction=junction_flow,
                                                length_km=float(str(df_connection.at[j, 'Length [m]']))/1000,
                                                diameter_m=float(str(df_connection.at[j, 'Diameter [mm]']))/1000,
                                                alpha_w_per_m2k=float(str(df_connection.at[j, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                                                text_k=int(temp_ext_k),
                                                name=str(df_connection.at[j,'Name'])+'_supply')

                    pp.create_pipe_from_parameters(net,
                                                from_junction=junction_return,
                                                to_junction=source_return,
                                                length_km=float(str(df_connection.at[j, 'Length [m]']))/1000,
                                                diameter_m=float(str(df_connection.at[j, 'Diameter [mm]']))/1000,
                                                alpha_w_per_m2k=float(str(df_connection.at[j, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                                                text_k=int(temp_ext_k),
                                                name=str(df_connection.at[j,'Name'])+'_supply')
            
    # Ericsson special case: Ericsson has only return connections;
    # Ericsson set as a slack node in the network.

    # Create auxiliary junction to connect to Ericsson
    Ericsson_row = df_nodetype.loc[df_nodetype['Name']=='Ericsson']
    Ericsson_geodata = (Ericsson_row.loc[:,'X-Coordinate'].values[0],Ericsson_row.loc[:,'Y-Coordinate'].values[0])
    g['Ericsson_connection'] = pp.create_junction(
                                        net, pn_bar=net_return_p_bar,
                                        tfluid_k=t_net_return_init_k,
                                        geodata=Ericsson_geodata,
                                        name='Ericsson_connection')

    # Tracker for Ericsson auxiliary junction connections
    e_pipe_tracker = 0

    for i in range(len(df_connection)):

        if str(df_connection.at[i, 'Start Node']) == 'Ericsson':
                    
            connection_name = g[str(df_connection.at[i, 'End Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_return']

            pp.create_pipe_from_parameters(net,
                from_junction=connection_name,
                to_junction=g['Ericsson_connection'],
                length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                alpha_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                text_k=int(temp_ext_k),
                name='Pipe_E_' + str(e_pipe_tracker)
                )
            
            e_pipe_tracker += 1
            
        elif str(df_connection.at[i, 'End Node']) == 'Ericsson':

            connection_name = g[str(df_connection.at[i, 'Start Node'])
                                            .replace("Junction-", "Junction_") + 
                                            '_return']

            pp.create_pipe_from_parameters(net,
                from_junction=connection_name,
                to_junction=g['Ericsson_connection'],
                length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                k_mm=.05,
                alpha_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                text_k=int(temp_ext_k),
                name='Pipe_E_' + str(i)
                )
            
            e_pipe_tracker += 1

    # Create Ericsson slack node

    Ericsson = pp.create_ext_grid(net,
                                junction=g['Ericsson_connection'],
                                p_bar=net_return_p_bar,
                                t_k=t_net_return_init_k,
                                name="Ericsson Ext. Grid")


    # List sources
    net.circ_pump_pressure
    #help(pp.create_pipe_from_parameters)

In [66]:
def createSink(col):
    ## Create pandapipes sink nodes

    # Desired return water temperature [K] = t[C] + 273.15
    # Use t_return_c = t_net_return_init_c to set to initial network temp
    t_return_c = t_net_return_init_c + 5
    t_return_k = t_return_c + 273.15

    #demand in kW
    col_name=df_sink.columns[col]
    # Get number of sinks and connections
    num_sinks = len(df_sink)
    num_connections = len(df_connection)

    # Tracker tracks number of sinks created
    sink_tracker = 0

    # Iterate over number of raw data rows to find and create sinks
    for i in range(num_sinks):

        sink_get = str(df_sink.at[i, 'Name'])
        sink_name=sink_get.replace('Junction-','Sink_')
        # Get sink details
        sink_type = str(df_sink.at[i, 'Category'])
        sink_priority = int(float(str(df_sink.at[i, 'Priority'])))
        sink_demand_kW = float(str(df_sink.at[i, col_name]))+float(str(df_sink.at[i, 'Base Demand [kW]']))
        mdot_kg_per_s_demand = float(str(df_sink.at[i, 'Base Demand [kg/s]']))
        sink_x = float(str(df_sink.at[i, 'X-Coordinate']))
        sink_y = float(str(df_sink.at[i, 'Y-Coordinate']))

        g[sink_get.replace('Junction-','Sink_')+'_supply']=pp.create_junction(net,
                                                                            pn_bar=net_flow_p_bar,
                                                                            tfluid_k=t_net_flow_init_k,
                                                                            geodata=(sink_x,sink_y),
                                                                            name=sink_get.replace('Junction-','Sink_') + '_supply')
        g[sink_get.replace('Junction-','Sink_')+'_return']=pp.create_junction(net,
                                                                            pn_bar=net_return_p_bar,
                                                                            tfluid_k=t_net_return_init_k,
                                                                            geodata=(sink_x,sink_y-100),
                                                                            name=sink_get.replace('Junction-','Sink_') + '_return')
        # Define sink
        pp.create_heat_consumer(net,
                                from_junction=g[sink_get.replace('Junction-','Sink_')+'_supply'],
                                to_junction=g[sink_get.replace('Junction-','Sink_')+'_return'],
                                diameter_m=43.1/1000,
                                qext_w=sink_demand_kW*1000,
                                # controlled_mdot_kg_per_s=mdot_kg_per_s_capacity,
                                #controlled_mdot_kg_per_s=mdot_kg_per_s_demand,
                                # deltat_k=50,
                                treturn_k=t_return_k,
                                name=sink_name, type=sink_type
                                )
        
        sink_flow = g[sink_get.replace('Junction-','Sink_')+'_supply']
        sink_return = g[sink_get.replace('Junction-','Sink_')+'_return']
        # Get connecting junction and pipe diameter
        for k in range(num_connections):
            if df_connection.at[k, "End Node"] == sink_get:
                junction_source = g[str(df_connection.at[k, 'Start Node']).
                                        replace("Junction-", "Junction_") + '_supply']
                junction_return = g[str(df_connection.at[k, 'Start Node']).
                                        replace("Junction-", "Junction_") + '_return']
                connection_d_mm = float(str(df_connection.at[k, 'Diameter [mm]']))
                mdot_kg_per_s_capacity = float(str(df_connection.at[k, 'Capacity [kg/s]']))
                
                pp.create_pipe_from_parameters(net,
                    from_junction=junction_source,
                    to_junction=sink_flow,
                    length_km=float(str(df_connection.at[k, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[k, 'Diameter [mm]']))/1000,
                    alpha_w_per_m2k=float(str(df_connection.at[k, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    text_k=int(temp_ext_k),
                    name=str(df_connection.at[k,'Name']+'_supply')
                    )
                pp.create_pipe_from_parameters(net,
                    from_junction=sink_return,
                    to_junction=junction_return,
                    length_km=float(str(df_connection.at[k, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[k, 'Diameter [mm]']))/1000,
                    alpha_w_per_m2k=float(str(df_connection.at[k, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    text_k=int(temp_ext_k),
                    name=str(df_connection.at[k,'Name']+'_return')
                    ) 

            elif df_connection.at[k, "Start Node"] == sink_get:
                junction_source = g[str(df_connection.at[k, 'End Node']).
                                        replace("Junction-", "Junction_") + '_supply']
                junction_return = g[str(df_connection.at[k, 'End Node']).
                                        replace("Junction-", "Junction_") + '_return']
                connection_d_mm = float(str(df_connection.at[k, 'Diameter [mm]']))
                mdot_kg_per_s_capacity = float(str(df_connection.at[k, 'Capacity [kg/s]']))
                pp.create_pipe_from_parameters(net,
                    from_junction=junction_source,
                    to_junction=sink_flow,
                    length_km=float(str(df_connection.at[k, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[k, 'Diameter [mm]']))/1000,
                    alpha_w_per_m2k=float(str(df_connection.at[k, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    text_k=int(temp_ext_k),
                    name=str(df_connection.at[k,'Name']+'_supply')
                    )
                pp.create_pipe_from_parameters(net,
                    from_junction=sink_return,
                    to_junction=junction_return,
                    length_km=float(str(df_connection.at[k, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[k, 'Diameter [mm]']))/1000,
                    alpha_w_per_m2k=float(str(df_connection.at[k, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    text_k=int(temp_ext_k),
                    name=str(df_connection.at[k,'Name']+'_return')
                    )  
            
        sink_tracker += 1

    #help(pp.create_heat_consumer)
    #print("Number of sinks created:", sink_tracker)
    net.heat_consumer

In [67]:
def createPipe():
    ## Create pipe connections

    # Defines pipe connections between junctions. Source and sink connections
    # are defined at heat pump node and heat exchanger creation.

    # Tracker tracks number of pipes created
    pipe_tracker = 0

    # Concatenate source and sink names to single NumPy array
    np_heaters_sinks = pd.concat([df_heater[['Name']], df_sink[['Name']]]).to_numpy()
    num_heaters_sinks = len(np_heaters_sinks)
    num_connections = len(df_connection)
    for i in range(num_connections):
        
        pipe_get = str(df_connection.at[i, 'Name']).replace("-", "_")

        start_node = str(df_connection.at[i, 'Start Node'])
        end_node = str(df_connection.at[i, 'End Node'])

        if str(df_connection.at[i, 'Has Supply Line']) == 'True': supply_line = True
        else: supply_line = False
        if str(df_connection.at[i, 'Has Return Line']) == 'True': return_line = True
        else: return_line = False
    
        # Exclude source and sink node connections (defined earlier)
        if start_node not in np_heaters_sinks and end_node not in np_heaters_sinks:
                
            # Create supply line pipe
            if supply_line == True:
                
                pipe_from = g[str(df_connection.at[i, 'Start Node']).
                                    replace("Junction-", "Junction_") + '_supply']
                pipe_to = g[str(df_connection.at[i, 'End Node']).
                                    replace("Junction-", "Junction_") + '_supply']
                pipe_name = pipe_get + '_supply'

                pp.create_pipe_from_parameters(net,
                    from_junction=pipe_from,
                    to_junction=pipe_to,
                    length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                    k_mm=.05,
                    alpha_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    sections=5,
                    text_k=int(temp_ext_k),
                    name=pipe_name
                    )
                pipe_tracker += 1
            
            # Create return line pipe
            if return_line == True:

                pipe_from = g[str(df_connection.at[i, 'End Node']).
                    replace("Junction-", "Junction_") + '_return']
                pipe_to = g[str(df_connection.at[i, 'Start Node']).
                    replace("Junction-", "Junction_") + '_return']
                pipe_name = pipe_get + '_return'

                pp.create_pipe_from_parameters(net,
                    from_junction=pipe_from,
                    to_junction=pipe_to,
                    length_km=float(str(df_connection.at[i, 'Length [m]']))/1000,
                    diameter_m=float(str(df_connection.at[i, 'Diameter [mm]']))/1000,
                    alpha_w_per_m2k=float(str(df_connection.at[i, 'Heat Transfer Coefficient [W/mK]'])), # W/mK from raw data
                    text_k=int(temp_ext_k),
                    name=pipe_name,
                    sections=5,
                    k_mm=.05
                    )
                pipe_tracker += 1

    #print("Number of pipes created: ", pipe_tracker)
    net.pipe

In [68]:
net.pipe.name

Series([], Name: name, dtype: object)

In [69]:
## Code for inspecting elements and debugging

# net.junction.loc(Junction_1000_return)

# Plotting
import pandapipes.plotting as plot

def netPlot():
    plot.simple_plot(net, plot_sinks=True, plot_sources=True,
                    junction_size=0.2, heat_consumer_size=0.5,
                    heat_consumer_color='Teal', pump_size=0.5,
                    pump_color='Blue')

In [70]:
def setParams(mid):
    global t_net_flow_init_k,t_out_k
    t_net_flow_init_k=mid-5
    t_out_k=mid

In [71]:
def run(col):
    ## Run pipeflow
    createJunction()
    createSource()
    createSink(col)
    createPipe()
    #netPlot()
    pp.pipeflow(net, mode='sequential')#,iter=10000) # Modes = hydraulics, sequential, bidirectional
    
    return net.res_circ_pump_pressure.at[0,'mdot_flow_kg_per_s']+net.res_circ_pump_pressure.at[1,'mdot_flow_kg_per_s']

In [72]:
#Initialisations
temp=120
g={}
net=pp.create_empty_network(name='Data',fluid='water')

setParams(temp+273.15)
#total_mdot_kg_per_s=abs(run(7))
total_mdot_kg_per_s=103.77407837111305+146.61465869121733
print(total_mdot_kg_per_s)
col=8
s=109
e=180.0
iter=0
mid=0
while s<e and iter<30:
    g={}
    net=pp.create_empty_network(name='Data',fluid='water')
    iter+=1
    mid=(s+e)/2.0
    setParams(mid+273.15)
    total_mdot_kg_per_s_INST=abs(run(col))
    if total_mdot_kg_per_s_INST>total_mdot_kg_per_s:
        s=mid+0.1
    elif total_mdot_kg_per_s_INST<total_mdot_kg_per_s:
        e=mid-0.1
    elif abs(total_mdot_kg_per_s_INST-total_mdot_kg_per_s)<=1.0:
        print(mid)
        print(f'\tObtained mdot {total_mdot_kg_per_s_INST}')
        print(f'\tReq mdot {total_mdot_kg_per_s}')
        break
    print(f'Temp = {mid}')
    print(f'\tObtained mdot {total_mdot_kg_per_s_INST}')
    print(f'\tReq mdot {total_mdot_kg_per_s}')


250.3887370623304
Temp = 144.5
	Obtained mdot 346.9407955189632
	Req mdot 250.3887370623304
Temp = 162.3
	Obtained mdot 293.7710792534799
	Req mdot 250.3887370623304
Temp = 171.2
	Obtained mdot 272.81139685483777
	Req mdot 250.3887370623304
Temp = 175.64999999999998
	Obtained mdot 263.40255590969895
	Req mdot 250.3887370623304
Temp = 177.875
	Obtained mdot 258.93450100322906
	Req mdot 250.3887370623304
Temp = 178.9875
	Obtained mdot 256.75613047791273
	Req mdot 250.3887370623304
Temp = 179.54375
	Obtained mdot 255.68045676279996
	Req mdot 250.3887370623304
Temp = 179.82187499999998
	Obtained mdot 255.14594901418917
	Req mdot 250.3887370623304
Temp = 179.9609375
	Obtained mdot 254.87952141412222
	Req mdot 250.3887370623304


In [73]:
# net.pipe.to_csv('pipe_inp.csv',index=False)

In [74]:
## Export model to JSON

pp.to_json(net, "output/models/RHN-Full1.json")

FileNotFoundError: [Errno 2] No such file or directory: 'output/models/RHN-Full1.json'

In [ ]:
net.res_heat_consumer.to_csv('mass_flow_allowable.csv', index = False) # Modes = res_heat_consumer, res_junction, res_pipe, res_circ_pump_pressure

In [ ]:
res_heat_consumer=net.res_heat_consumer
mdot_kg_per_s_ARR=res_heat_consumer['mdot_from_kg_per_s']
print(mdot_kg_per_s_ARR)

0       0.000000
1       0.326717
2       0.352318
3       0.252352
4       0.332812
5       0.305992
6       0.088994
7       0.310868
8       0.442530
9       0.254790
10      0.393767
11      0.514457
12      0.453502
13      0.399862
14      0.485199
15      0.074365
16      0.502266
17      0.302335
18      0.156044
19      0.214560
20      0.036573
21      3.802347
22      0.318183
23      0.063393
24      0.938701
25      0.340127
26      0.026820
27      0.058516
28      0.020725
29      0.047545
30      0.031696
31      0.026820
32      0.020725
33      0.031696
34      0.140196
35      0.031696
36      4.403359
37      0.636366
38      0.577850
39      0.368166
40      0.323059
41      0.887499
42      0.031696
43      0.599793
44      0.137757
45      0.169454
46      0.063393
47      0.430339
48      0.431558
49      0.058516
50      0.458379
51      0.026820
52      0.031696
53      0.020725
54      0.031696
55      0.036573
56      0.466912
57      0.020725
58      0.5559

In [ ]:
#Creation of a tree structure from excel data.

#roots of tree are the heaters except Ericsson
starts=[]
for i in range(len(df_heater)):
    if df_heater.at[i,'Name']!='Ericsson':
        starts.append(df_heater.at[i,'Name'])

#Goes through the df_nodetypr to store all nodes in a dict which will have their corresponding parent & children
nodes = {x: {'parent': [], 'children': {}} for x in df_nodetype['Name']}
nodes['Sarfvik']['parent']='-1'
nodes['Kirkkonummi']['parent']='-1'
vis={x: {x:-1} for x in df_nodetype['Name']}
vis['Sarfvik']=1
vis['Kirkkonummi']=1

#Here we add connections to a dictionary so as to create tree by assigning parents and children. 
    #If a connection has 'Ericsson' has one end then atleast one of 'Supply' or 'Return' should be 'True'
    #If its a connection between two nodes where none of them is 'Ericsson' then both 'Supply' and 'Return' should be 'True'.
filtered_df_connection = df_connection[
    ((df_connection['Start Node'] == 'Ericsson') | (df_connection['End Node'] == 'Ericsson')) &
    ((df_connection['Has Supply Line'] == True) | (df_connection['Has Return Line'] == True))
    |
    (~df_connection['Start Node'].eq('Ericsson') & ~df_connection['End Node'].eq('Ericsson') &
    (df_connection['Has Supply Line'] == True) & (df_connection['Has Return Line'] == True))
]

#This dictionary 'connections' has all the connections between all valid nodes. Valid connections are filtered above
    #Ericssion cases handled properly
connections = filtered_df_connection[['Start Node', 'End Node', 'Name']].values.tolist()

## Note : It has been assumed that no node has more than one parent. 
    #Since heaters are roots of the tree, two or more heaters cannot be connected to a single node (junction)
## Note : It is also assumed that there is one connection (one return and one supply) between any two nodes(junction)
bfs=[x for x in starts]
level=[0 for x in starts]
i=0
count=0
while i<len(bfs):
    j=0
    if bfs[i]=='Ericsson':
        count+=1
        i+=1
        continue
    while j<len(connections):
        connection=connections[j]
        if bfs[i]==connection[0]:
            if connection[1] in bfs:
                print(f"loop found {connection[0]}   {connection[1]}")
            bfs.append(connection[1])
            vis[connection[1]]=1
            level.append(level[i]+1)
            nodes[bfs[i]]['children'][connection[1]]=connection[2]
            nodes[connection[1]]['parent'].append(bfs[i])
            connections.pop(j)
        elif bfs[i]==connection[1]:
            if connection[0] in bfs:
                print(f"loop found {connection[1]}   {connection[0]}")
            bfs.append(connection[0])
            vis[connection[0]]=1
            level.append(level[i]+1)
            nodes[bfs[i]]['children'][connection[0]]=connection[2]
            nodes[connection[0]]['parent'].append(bfs[i])
            connections.pop(j)
        else:
            j+=1
    i+=1
print(count)
#Visualizing the tree
curr_level=0
idx=0
for ele in bfs:
    if curr_level==level[idx]:
        print(ele,end=" ")
    else:
        curr_level=level[idx]
        print()
        print(ele,end=" ")
    idx+=1
print("\n\n\nNodes:\nName\t\tparent\t\tChildren\n")
for node in nodes:
    print(node,nodes[node]['parent'],nodes[node]['children'],sep="          ")

loop found Junction-101660   Ericsson
loop found Junction-22175   Junction-36301
loop found Junction-27498   Junction-35697
loop found Junction-127219   Junction-129143
loop found Junction-20306   Junction-19701
loop found Junction-9802   Junction-99110
loop found Junction-35794   Junction-106504
loop found Junction-31788   Junction-32382
loop found Junction-55565   Junction-99693
loop found Junction-17278   Junction-18843
loop found Junction-18008   Junction-17277
loop found Junction-2237   Junction-18841
loop found Junction-2098   Junction-18017
loop found Junction-18841   Junction-2228
loop found Junction-18021   Junction-2091
loop found Junction-2009   Junction-2007
loop found Junction-2006   Junction-2007
loop found Junction-2006   Junction-18027
loop found Junction-39069   Junction-39122
loop found Junction-70718   Junction-37764
2
Sarfvik Kirkkonummi 
Junction-6737 Junction-2040 
Junction-6630 Junction-128917 Junction-2039 Junction-116892 
Junction-7838 Junction-127510 Junction-

In [ ]:
# #Creation of a tree structure from excel data.

# #roots of tree are the heaters except Ericsson
# starts=[]
# for i in range(len(net.circ_pump_pressure)):
#     if (net.circ_pump_pressure).at[i,'name']!='Ericsson':
#         starts.append((net.circ_pump_pressure).at[i,'name'])

# bfs=[]
# level=[]
# vis=[]
# i=0
# while i<len(net.circ_pump_pressure):
#     name=net.circ_pump_pressure.at[i,'name']
#     return_junction=net.circ_pump_pressure.at[i,'return_junction']
#     flow_junction=net.circ_pump_pressure.at[i,'flow_junction']
#     if net.junction.at[return_junction,'name'].endswith('_return') and net.junction.at[flow_junction,'name'].endswith('_supply'):
#         bfs.append([flow_junction,-1])
#         level.append(0)
#     i+=1

# i=0
# while i<len(bfs):
#     vis.append(bfs[i][0])
#     k=0
#     while k<len(net.pipe):
#         from_junction=net.pipe.at[k,'from_junction']
#         to_junction=net.pipe.at[k,'to_junction']
#         if from_junction==bfs[i][0]:
#             if to_junction in vis:
#                 print('loop found',end="")
#             else:
#                 bfs.append([to_junction,from_junction])
#                 level.append(level[i]+1)
#         elif to_junction==bfs[i][0]:
#             if from_junction in vis:
#                 print('loop found',end="")
#             else:
#                 bfs.append([from_junction,to_junction])
#                 level.append(level[i]+1)
#         k+=1
#     i+=1

# #Visualizing the tree
# print(starts)
# curr_lev=1
# for i in range(len(starts),len(bfs)):
#     if curr_lev==level[i]:
#         print(bfs[i],end=" ")
#     else:
#         curr_lev+=1
#         print()
#         print(bfs[i],end=" ")

In [ ]:
#Accommodating changes of network

"""
    1. Change in sink demand
    2. Leak/removal of pipes
    3. Addtion/subtraction of sink(user)
    4. Addition of storage tank
    5. Changes in environment condition
"""

'\n    1. Change in sink demand\n    2. Leak/removal of pipes\n    3. Addtion/subtraction of sink(user)\n    4. Addition of storage tank\n    5. Changes in environment condition\n'